In [1]:
import sys
sys.path.append('/home/luca/GitRepositories/Brancher')

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from brancher.variables import ProbabilisticModel

from brancher.standard_variables import NormalVariable 
from brancher import inference

$ f \sim GP\! \left(\mu(t), k(t,t') \right) $

$ y_j \sim N\!\left(f(x_j), \sigma \right) $

In [ ]:
# Model
mu = ConstantMean(0.)
cov = SquaredExponential(scale=0.2)
f = GP(mu, cov, name="f")
y = Normal(f(x), 0.2, name="y")

In [ ]:
num_datapoints = 20
x_range = np.linspace(-2, 2, num_datapoints)
x = DeterministicVariable(x_range, name="x")

In [ ]:
fx = f(x)

In [ ]:
y = Normal(f(x), 0.2, name="y")